In [ ]:
# Force environment activation and GPU configuration
import os
import sys
import subprocess

# Method 1: Try to activate conda environment programmatically
def activate_trading_env():
    """Attempt to activate trading-env within Python."""
    try:
        # Get conda info
        result = subprocess.run(['conda', 'info', '--base'], capture_output=True, text=True)
        conda_base = result.stdout.strip()
        
        # Add conda env to path
        trading_env_path = f"{conda_base}/envs/trading-env"
        if os.path.exists(trading_env_path):
            # Prepend to PATH
            env_bin = f"{trading_env_path}/bin"
            if env_bin not in os.environ.get('PATH', ''):
                os.environ['PATH'] = f"{env_bin}:{os.environ.get('PATH', '')}"
            
            # Set conda environment variables
            os.environ['CONDA_DEFAULT_ENV'] = 'trading-env'
            os.environ['CONDA_PREFIX'] = trading_env_path
            
            print(f"✅ Activated trading-env: {trading_env_path}")
            return True
    except Exception as e:
        print(f"⚠️ Could not activate trading-env: {e}")
    
    return False

# Method 2: Import TensorFlow from correct environment
def import_tensorflow_gpu():
    """Import TensorFlow with GPU support."""
    try:
        # Try importing TensorFlow
        import tensorflow as tf
        
        print(f"TensorFlow version: {tf.__version__}")
        print(f"TensorFlow location: {tf.__file__}")
        
        # Check if it's from the correct environment
        if 'trading-env' in tf.__file__:
            print("✅ Using TensorFlow from trading-env")
        else:
            print("⚠️ TensorFlow not from trading-env")
            
        return tf
        
    except ImportError as e:
        print(f"❌ TensorFlow import failed: {e}")
        print("Installing TensorFlow...")
        subprocess.run([sys.executable, '-m', 'pip', 'install', 'tensorflow'])
        import tensorflow as tf
        return tf

# Activate environment and import TensorFlow
activate_trading_env()
tf = import_tensorflow_gpu()

# Enhanced GPU configuration with fallback detection
def configure_gpu_with_fallback():
    """Configure GPU with enhanced detection and fallback options."""
    print("🔧 Configuring GPU with enhanced detection...")
    
    # Method 1: Standard TensorFlow GPU detection
    gpus = tf.config.list_physical_devices('GPU')
    
    # Method 2: Force CUDA device detection if standard method fails
    if not gpus:
        print("Standard GPU detection failed, trying CUDA detection...")
        try:
            # Force CUDA initialization
            with tf.device('/GPU:0'):
                test_tensor = tf.constant([1.0])
                print("CUDA device accessible")
                gpus = ['/GPU:0']  # Manual GPU specification
        except:
            print("CUDA detection also failed")
    
    if gpus:
        print(f"🎮 Found {len(gpus)} GPU(s):")
        
        try:
            # Configure each GPU
            for i, gpu in enumerate(gpus):
                print(f"  GPU {i}: {gpu}")
                
                if hasattr(tf.config.experimental, 'set_memory_growth'):
                    if isinstance(gpu, str):
                        # Manual GPU reference
                        print(f"    Manual GPU configuration for {gpu}")
                    else:
                        # Standard PhysicalDevice
                        tf.config.experimental.set_memory_growth(gpu, True)
                        print(f"    ✅ Memory growth enabled")
                        
                        # Get GPU details
                        try:
                            details = tf.config.experimental.get_device_details(gpu)
                            print(f"    Details: {details}")
                        except:
                            pass
            
            # Set mixed precision
            try:
                policy = tf.keras.mixed_precision.Policy('mixed_float16')
                tf.keras.mixed_precision.set_global_policy(policy)
                print("  ✅ Mixed precision enabled (float16)")
            except Exception as e:
                print(f"  ⚠️ Mixed precision failed: {e}")
            
            # Test GPU computation
            try:
                with tf.device('/GPU:0'):
                    a = tf.random.normal([1000, 1000])
                    b = tf.random.normal([1000, 1000])
                    c = tf.matmul(a, b)
                    print(f"  ✅ GPU computation test: {c.device}")
            except Exception as e:
                print(f"  ⚠️ GPU computation test failed: {e}")
            
            return True
            
        except Exception as e:
            print(f"❌ GPU configuration failed: {e}")
            
    else:
        print("❌ No GPUs detected")
        print("Diagnostic information:")
        print(f"  Python executable: {sys.executable}")
        print(f"  TensorFlow built with CUDA: {tf.test.is_built_with_cuda()}")
        
        # Try nvidia-smi check
        try:
            result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
            if result.returncode == 0:
                print("  ✅ nvidia-smi works (GPU hardware available)")
            else:
                print("  ❌ nvidia-smi failed")
        except:
            print("  ❌ nvidia-smi not available")
    
    return False

# Run GPU configuration
gpu_available = configure_gpu_with_fallback()

if gpu_available:
    print("\n⚡ GPU Configuration Complete!")
    print("  - GPU memory growth enabled")
    print("  - Mixed precision training ready")
    print("  - Ready for hyperparameter optimization")
    
    # Enable XLA if possible
    try:
        tf.config.optimizer.set_jit(True)
        print("  - XLA compilation enabled")
    except:
        print("  - XLA compilation not available")
        
else:
    print("\n🖥️ Using CPU Configuration:")
    print("  - Multi-threading enabled")
    print("  - CPU optimization active")
    
    # Optimize for CPU
    tf.config.threading.set_intra_op_parallelism_threads(0)
    tf.config.threading.set_inter_op_parallelism_threads(0)

print(f"\n📊 Final Status:")
print(f"  TensorFlow: {tf.__version__}")
print(f"  GPU devices: {len(tf.config.list_physical_devices('GPU'))}")
print(f"  Ready for optimization: {'GPU' if gpu_available else 'CPU'}")

# 🎯 Hyperparameter Optimization for CNN-LSTM Trading Strategy

This notebook optimizes hyperparameters for the CNN-LSTM forex trading model using Optuna.

## Optimization Targets
- **Model Architecture**: Conv1D filters, LSTM units, layers
- **Training Parameters**: Learning rate, batch size, epochs
- **Regularization**: Dropout rates, L1/L2 regularization
- **Data Parameters**: Lookback window, feature selection
- **Trading Parameters**: Confidence thresholds, signal generation

## Objective Functions
- Primary: Sharpe Ratio (risk-adjusted returns)
- Secondary: Total Return, Maximum Drawdown, Win Rate
- Model: Validation Accuracy, Loss

## Search Strategy
- Multi-objective optimization
- Bayesian optimization with TPE sampler
- Early pruning of poor trials
- Cross-validation for robustness

# Suppress TensorFlow warnings AFTER GPU configuration
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Configuration
DATA_PATH = "RCS_CNN_LSTM_Notebook/RCS_CNN_LSTM_Notebook/data"
SYMBOLS = ['EURUSD', 'GBPUSD', 'USDJPY', 'AUDUSD', 'USDCAD', 'EURJPY', 'GBPJPY']
TARGET_SYMBOL = 'EURUSD'  # Focus on one symbol for optimization

# Optimization settings
N_TRIALS = 100  # Number of optimization trials
N_JOBS = 1      # Parallel jobs (set to 1 for stability)
TIMEOUT = 3600  # 1 hour timeout
CV_SPLITS = 3   # Cross-validation splits

print(f"Target symbol for optimization: {TARGET_SYMBOL}")
print(f"Optimization trials: {N_TRIALS}")
print(f"CV splits: {CV_SPLITS}")

# Verify GPU is still available after configuration
print(f"\n🔍 GPU Status Check:")
print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")
print(f"GPU available: {tf.test.is_gpu_available()}")
if tf.config.list_physical_devices('GPU'):
    print(f"GPU name: {tf.test.gpu_device_name()}")
    print("✅ GPU acceleration ready for hyperparameter optimization")

In [ ]:
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import pickle
import json
from datetime import datetime
import logging

warnings.filterwarnings('ignore')

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Install optuna if not available
try:
    import optuna
    from optuna.samplers import TPESampler
    from optuna.pruners import MedianPruner
    print("✅ Optuna available")
except ImportError:
    print("Installing Optuna...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "optuna"])
    import optuna
    from optuna.samplers import TPESampler
    from optuna.pruners import MedianPruner
    print("✅ Optuna installed and imported")

# Add src path
src_path = os.path.join(os.getcwd(), 'RCS_CNN_LSTM_Notebook', 'RCS_CNN_LSTM_Notebook', 'src')
if src_path not in sys.path:
    sys.path.append(src_path)

# Import ML libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1_l2
from tensorflow.keras.optimizers import Adam

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import accuracy_score

# Technical analysis
import ta
from ta.volatility import BollingerBands, AverageTrueRange
from ta.trend import ADXIndicator, MACD, CCIIndicator
from ta.momentum import StochasticOscillator, ROCIndicator, RSIIndicator

# Suppress TensorFlow warnings
tf.get_logger().setLevel('ERROR')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Configuration
DATA_PATH = "RCS_CNN_LSTM_Notebook/RCS_CNN_LSTM_Notebook/data"
SYMBOLS = ['EURUSD', 'GBPUSD', 'USDJPY', 'AUDUSD', 'USDCAD', 'EURJPY', 'GBPJPY']
TARGET_SYMBOL = 'EURUSD'  # Focus on one symbol for optimization

# Optimization settings
N_TRIALS = 100  # Number of optimization trials
N_JOBS = 1      # Parallel jobs (set to 1 for stability)
TIMEOUT = 3600  # 1 hour timeout
CV_SPLITS = 3   # Cross-validation splits

print(f"Target symbol for optimization: {TARGET_SYMBOL}")
print(f"Optimization trials: {N_TRIALS}")
print(f"CV splits: {CV_SPLITS}")

## 2. Load and Prepare Base Data

In [ ]:
def load_all_data():
    """
    Load all data once for optimization.
    """
    print("📥 Loading base data...")
    
    # Load parquet files
    data = {}
    for symbol in SYMBOLS:
        parquet_file = os.path.join(DATA_PATH, f"metatrader_{symbol}.parquet")
        if os.path.exists(parquet_file):
            df = pd.read_parquet(parquet_file)
            
            # Standardize time column
            if 'timestamp' in df.columns:
                df['time'] = pd.to_datetime(df['timestamp'])
            elif 'time' in df.columns:
                df['time'] = pd.to_datetime(df['time'])
            
            df = df.set_index('time')
            
            # Add volume if missing
            if 'tick_volume' not in df.columns:
                df['tick_volume'] = df.get('volume', 0)
            
            # Store in MultiIndex format
            for col in ['open', 'high', 'low', 'close', 'tick_volume']:
                if col in df.columns:
                    data[(symbol, col)] = df[col]
    
    prices_df = pd.DataFrame(data)
    print(f"✅ Loaded data: {prices_df.shape}")
    return prices_df

def calculate_all_indicators(prices_df, symbols):
    """
    Calculate technical indicators for all symbols.
    """
    print("📊 Calculating technical indicators...")
    
    all_indicators = {}
    
    for symbol in symbols:
        if (symbol, 'close') not in prices_df.columns:
            continue
            
        close = prices_df[(symbol, 'close')].dropna()
        high = prices_df[(symbol, 'high')].dropna() if (symbol, 'high') in prices_df.columns else close
        low = prices_df[(symbol, 'low')].dropna() if (symbol, 'low') in prices_df.columns else close
        
        indicators = pd.DataFrame(index=close.index)
        
        # Comprehensive indicator set
        try:
            # Momentum
            indicators['rsi'] = RSIIndicator(close=close, window=14).rsi()
            indicators['rsi_fast'] = RSIIndicator(close=close, window=7).rsi()
            indicators['rsi_slow'] = RSIIndicator(close=close, window=21).rsi()
            indicators['roc'] = ROCIndicator(close=close, window=10).roc()
            indicators['roc_fast'] = ROCIndicator(close=close, window=5).roc()
            
            # Trend
            macd = MACD(close=close)
            indicators['macd'] = macd.macd()
            indicators['macd_signal'] = macd.macd_signal()
            indicators['macd_histogram'] = macd.macd_diff()
            
            indicators['cci'] = CCIIndicator(high=high, low=low, close=close).cci()
            indicators['adx'] = ADXIndicator(high=high, low=low, close=close).adx()
            
            # Volatility
            indicators['atr'] = AverageTrueRange(high=high, low=low, close=close).average_true_range()
            indicators['atr_norm'] = indicators['atr'] / close
            
            bb = BollingerBands(close=close)
            indicators['bb_upper'] = bb.bollinger_hband()
            indicators['bb_lower'] = bb.bollinger_lband()
            indicators['bb_width'] = bb.bollinger_wband()
            indicators['bb_position'] = (close - bb.bollinger_lband()) / (bb.bollinger_hband() - bb.bollinger_lband())
            
            # Stochastic
            stoch = StochasticOscillator(high=high, low=low, close=close)
            indicators['stoch_k'] = stoch.stoch()
            indicators['stoch_d'] = stoch.stoch_signal()
            
            # Price features
            indicators['return_1'] = close.pct_change(1)
            indicators['return_3'] = close.pct_change(3)
            indicators['return_5'] = close.pct_change(5)
            indicators['return_10'] = close.pct_change(10)
            
            # Moving averages
            indicators['sma_5'] = close.rolling(5).mean()
            indicators['sma_10'] = close.rolling(10).mean()
            indicators['sma_20'] = close.rolling(20).mean()
            indicators['sma_50'] = close.rolling(50).mean()
            
            indicators['ema_5'] = close.ewm(span=5).mean()
            indicators['ema_10'] = close.ewm(span=10).mean()
            indicators['ema_20'] = close.ewm(span=20).mean()
            
            # Volatility measures
            indicators['volatility_5'] = close.rolling(5).std()
            indicators['volatility_10'] = close.rolling(10).std()
            indicators['volatility_20'] = close.rolling(20).std()
            
            # Price position
            indicators['price_position_5'] = (close - close.rolling(5).min()) / (close.rolling(5).max() - close.rolling(5).min())
            indicators['price_position_20'] = (close - close.rolling(20).min()) / (close.rolling(20).max() - close.rolling(20).min())
            
            # Time features
            indicators['hour'] = indicators.index.hour
            indicators['day_of_week'] = indicators.index.dayofweek
            indicators['month'] = indicators.index.month
            
            # Clean data
            indicators = indicators.fillna(method='ffill').fillna(method='bfill').fillna(0)
            indicators = indicators.replace([np.inf, -np.inf], 0)
            
            all_indicators[symbol] = indicators
            print(f"✅ {symbol}: {indicators.shape[1]} indicators")
            
        except Exception as e:
            print(f"❌ Failed to calculate indicators for {symbol}: {e}")
            continue
    
    return all_indicators

def calculate_rcs(prices_df, symbols):
    """
    Calculate Relative Currency Strength.
    """
    print("🧮 Calculating RCS...")
    
    close_prices = {}
    for symbol in symbols:
        if (symbol, 'close') in prices_df.columns:
            close_prices[symbol] = prices_df[(symbol, 'close')]
    
    if not close_prices:
        return pd.DataFrame()
    
    close_df = pd.DataFrame(close_prices)
    log_returns = np.log(close_df / close_df.shift(1)).dropna()
    
    currencies = list(set([s[:3] for s in close_prices.keys()] + [s[3:6] for s in close_prices.keys()]))
    
    rcs_data = {c: [] for c in currencies}
    
    for i in range(len(log_returns)):
        row = log_returns.iloc[i]
        daily_strength = {c: 0 for c in currencies}
        counts = {c: 0 for c in currencies}
        
        for pair, ret in row.items():
            if pd.notna(ret):
                base, quote = pair[:3], pair[3:]
                daily_strength[base] += ret
                daily_strength[quote] -= ret
                counts[base] += 1
                counts[quote] += 1
        
        for c in currencies:
            avg = daily_strength[c] / counts[c] if counts[c] else 0
            rcs_data[c].append(avg)
    
    rcs_df = pd.DataFrame(rcs_data, index=log_returns.index)
    print(f"✅ RCS calculated: {rcs_df.shape}")
    
    return rcs_df

# Load base data
print("Loading base dataset...")
prices = load_all_data()
all_indicators = calculate_all_indicators(prices, SYMBOLS)
rcs = calculate_rcs(prices, SYMBOLS)

print(f"\n✅ Base data loaded:")
print(f"  Prices: {prices.shape}")
print(f"  Indicators: {len(all_indicators)} symbols")
print(f"  RCS: {rcs.shape}")

## 3. Define Hyperparameter Search Space

In [ ]:
def suggest_hyperparameters(trial):
    """
    Define the hyperparameter search space.
    """
    params = {
        # Data parameters
        'lookback_window': trial.suggest_int('lookback_window', 10, 50),
        'feature_selection_method': trial.suggest_categorical('feature_selection_method', 
                                                            ['all', 'top_correlation', 'variance_threshold', 'rfe']),
        'max_features': trial.suggest_int('max_features', 10, 30),
        
        # Model architecture
        'conv1d_filters_1': trial.suggest_int('conv1d_filters_1', 32, 128, step=16),
        'conv1d_filters_2': trial.suggest_int('conv1d_filters_2', 16, 64, step=8),
        'conv1d_kernel_size': trial.suggest_int('conv1d_kernel_size', 2, 5),
        'lstm_units': trial.suggest_int('lstm_units', 25, 100, step=5),
        'dense_units': trial.suggest_int('dense_units', 10, 50, step=5),
        
        # Regularization
        'dropout_rate': trial.suggest_float('dropout_rate', 0.1, 0.5),
        'l1_reg': trial.suggest_float('l1_reg', 1e-5, 1e-2, log=True),
        'l2_reg': trial.suggest_float('l2_reg', 1e-5, 1e-2, log=True),
        
        # Training parameters
        'learning_rate': trial.suggest_float('learning_rate', 1e-4, 1e-2, log=True),
        'batch_size': trial.suggest_categorical('batch_size', [16, 32, 64, 128]),
        'epochs': trial.suggest_int('epochs', 50, 200),
        'patience': trial.suggest_int('patience', 5, 20),
        
        # Trading parameters
        'confidence_threshold_high': trial.suggest_float('confidence_threshold_high', 0.55, 0.8),
        'confidence_threshold_low': trial.suggest_float('confidence_threshold_low', 0.2, 0.45),
    }
    
    # Ensure threshold consistency
    if params['confidence_threshold_low'] >= params['confidence_threshold_high']:
        params['confidence_threshold_low'] = params['confidence_threshold_high'] - 0.1
    
    return params

print("✅ Hyperparameter search space defined")
print("\nSearch space summary:")
print("- Lookback window: 10-50")
print("- Conv1D filters: 32-128, 16-64")
print("- LSTM units: 25-100")
print("- Learning rate: 1e-4 to 1e-2")
print("- Batch size: 16, 32, 64, 128")
print("- Dropout: 0.1-0.5")
print("- Confidence thresholds: optimized")

## 4. Feature Selection Functions

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif, VarianceThreshold, RFE
from sklearn.ensemble import RandomForestClassifier

def select_features(features_df, target, method='all', max_features=20):
    """
    Select features based on specified method.
    """
    if method == 'all':
        return features_df.columns.tolist()
    
    elif method == 'top_correlation':
        # Select features with highest correlation to target
        correlations = abs(features_df.corrwith(target))
        top_features = correlations.nlargest(max_features).index.tolist()
        return top_features
    
    elif method == 'variance_threshold':
        # Remove low variance features first
        selector = VarianceThreshold(threshold=0.01)
        selector.fit(features_df)
        selected_features = features_df.columns[selector.get_support()].tolist()
        
        # If still too many, use correlation
        if len(selected_features) > max_features:
            correlations = abs(features_df[selected_features].corrwith(target))
            selected_features = correlations.nlargest(max_features).index.tolist()
        
        return selected_features
    
    elif method == 'rfe':
        # Recursive Feature Elimination with RandomForest
        rf = RandomForestClassifier(n_estimators=50, random_state=42)
        selector = RFE(rf, n_features_to_select=min(max_features, len(features_df.columns)))
        selector.fit(features_df, target)
        selected_features = features_df.columns[selector.support_].tolist()
        return selected_features
    
    else:
        return features_df.columns.tolist()

print("✅ Feature selection functions defined")

## 5. Model Building and Training Functions

In [ ]:
def create_optimized_model(input_shape, params):
    """
    Create model with optimized hyperparameters using GPU.
    """
    # Force GPU device placement if available
    with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
        model = Sequential()
        
        # First Conv1D layer
        model.add(Conv1D(
            filters=params['conv1d_filters_1'],
            kernel_size=params['conv1d_kernel_size'],
            activation='relu',
            input_shape=input_shape,
            kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])
        ))
        model.add(BatchNormalization())
        model.add(Dropout(params['dropout_rate']))
        
        # Second Conv1D layer
        model.add(Conv1D(
            filters=params['conv1d_filters_2'],
            kernel_size=params['conv1d_kernel_size'],
            activation='relu',
            kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])
        ))
        model.add(BatchNormalization())
        model.add(Dropout(params['dropout_rate']))
        
        # LSTM layer
        model.add(LSTM(
            units=params['lstm_units'],
            kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg']),
            recurrent_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])
        ))
        model.add(BatchNormalization())
        model.add(Dropout(params['dropout_rate']))
        
        # Dense layer (optional)
        if params['dense_units'] > 0:
            model.add(Dense(
                units=params['dense_units'],
                activation='relu',
                kernel_regularizer=l1_l2(l1=params['l1_reg'], l2=params['l2_reg'])
            ))
            model.add(Dropout(params['dropout_rate']))
        
        # Output layer
        model.add(Dense(1, activation='sigmoid'))
        
        # Compile with optimized parameters
        optimizer = Adam(learning_rate=params['learning_rate'])
        model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

def prepare_training_data(symbol, indicators, rcs_df, prices_df, params):
    """
    Prepare training data with optimized parameters.
    """
    # Create target
    close_prices = prices_df[(symbol, 'close')]
    target = (close_prices.shift(-1) > close_prices).astype(int).dropna()
    
    # Combine features
    features = indicators.copy()
    
    # Add RCS features if available
    if not rcs_df.empty:
        for currency in ['USD', 'EUR', 'GBP', 'JPY']:
            if currency in rcs_df.columns:
                rcs_aligned = rcs_df[currency].reindex(features.index, method='ffill')
                features[f'rcs_{currency}'] = rcs_aligned
    
    # Align features and target
    common_index = features.index.intersection(target.index)
    features = features.loc[common_index]
    target = target.loc[common_index]
    
    # Clean features
    features = features.fillna(method='ffill').fillna(method='bfill').fillna(0)
    features = features.replace([np.inf, -np.inf], 0)
    
    # Feature selection
    selected_features = select_features(
        features, target, 
        method=params['feature_selection_method'],
        max_features=params['max_features']
    )
    
    features_selected = features[selected_features]
    
    # Scale features
    scaler = StandardScaler()
    features_scaled = scaler.fit_transform(features_selected)
    
    # Create sequences
    lookback = params['lookback_window']
    X, y = [], []
    
    for i in range(lookback, len(features_scaled)):
        X.append(features_scaled[i-lookback:i])
        y.append(target.iloc[i])
    
    X = np.array(X)
    y = np.array(y)
    
    return X, y, selected_features, scaler

print("✅ Model building functions defined with GPU device placement")

## 6. Evaluation and Objective Functions

In [ ]:
def calculate_trading_metrics(y_true, y_pred_proba, returns, params):
    """
    Calculate trading performance metrics.
    """
    # Generate trading signals
    signals = np.where(
        y_pred_proba > params['confidence_threshold_high'], 1,
        np.where(y_pred_proba < params['confidence_threshold_low'], -1, 0)
    )
    
    # Calculate strategy returns
    strategy_returns = signals * returns
    
    # Performance metrics
    total_return = np.sum(strategy_returns)
    
    # Sharpe ratio (annualized)
    if np.std(strategy_returns) > 0:
        sharpe_ratio = np.sqrt(252 * 24) * np.mean(strategy_returns) / np.std(strategy_returns)
    else:
        sharpe_ratio = 0
    
    # Maximum drawdown
    cumulative_returns = np.cumsum(strategy_returns)
    running_max = np.maximum.accumulate(cumulative_returns)
    drawdowns = cumulative_returns - running_max
    max_drawdown = np.min(drawdowns) if len(drawdowns) > 0 else 0
    
    # Win rate
    total_trades = np.sum(signals != 0)
    winning_trades = np.sum(strategy_returns > 0)
    win_rate = winning_trades / total_trades if total_trades > 0 else 0
    
    # Return metrics
    return {
        'total_return': total_return,
        'sharpe_ratio': sharpe_ratio,
        'max_drawdown': max_drawdown,
        'win_rate': win_rate,
        'total_trades': total_trades
    }

def cross_validate_model(X, y, params, n_splits=3):
    """
    Perform time series cross-validation with GPU acceleration.
    """
    tscv = TimeSeriesSplit(n_splits=n_splits)
    
    cv_scores = []
    cv_sharpe_ratios = []
    
    for fold, (train_idx, val_idx) in enumerate(tscv.split(X)):
        X_train, X_val = X[train_idx], X[val_idx]
        y_train, y_val = y[train_idx], y[val_idx]
        
        # Create and train model with GPU
        with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
            model = create_optimized_model((X.shape[1], X.shape[2]), params)
            
            # Early stopping
            early_stopping = EarlyStopping(
                monitor='val_loss',
                patience=params['patience'],
                restore_best_weights=True,
                verbose=0
            )
            
            # Train model
            history = model.fit(
                X_train, y_train,
                validation_data=(X_val, y_val),
                epochs=params['epochs'],
                batch_size=params['batch_size'],
                callbacks=[early_stopping],
                verbose=0
            )
        
        # Evaluate
        y_pred_proba = model.predict(X_val, verbose=0).flatten()
        y_pred = (y_pred_proba > 0.5).astype(int)
        
        # Model accuracy
        accuracy = accuracy_score(y_val, y_pred)
        cv_scores.append(accuracy)
        
        # Trading performance (simplified - assume unit returns)
        returns = np.random.normal(0, 0.001, len(y_val))  # Placeholder returns
        trading_metrics = calculate_trading_metrics(y_val, y_pred_proba, returns, params)
        cv_sharpe_ratios.append(trading_metrics['sharpe_ratio'])
        
        # Clear model to save memory
        del model
        tf.keras.backend.clear_session()
    
    return {
        'mean_accuracy': np.mean(cv_scores),
        'std_accuracy': np.std(cv_scores),
        'mean_sharpe': np.mean(cv_sharpe_ratios),
        'std_sharpe': np.std(cv_sharpe_ratios)
    }

print("✅ Evaluation functions defined with GPU acceleration")

## 7. Optimization Objective Function

In [ ]:
def objective(trial):
    """
    Objective function for Optuna optimization.
    """
    try:
        # Get hyperparameters
        params = suggest_hyperparameters(trial)
        
        # Prepare data
        X, y, selected_features, scaler = prepare_training_data(
            TARGET_SYMBOL, 
            all_indicators[TARGET_SYMBOL], 
            rcs, 
            prices, 
            params
        )
        
        # Check if we have enough data
        if len(X) < 100:
            return -1.0  # Penalize insufficient data
        
        # Perform cross-validation
        cv_results = cross_validate_model(X, y, params, n_splits=CV_SPLITS)
        
        # Multi-objective optimization
        # Primary: Sharpe ratio (risk-adjusted returns)
        # Secondary: Accuracy (model performance)
        
        sharpe_score = cv_results['mean_sharpe']
        accuracy_score = cv_results['mean_accuracy']
        
        # Combined objective (weighted)
        # Prioritize Sharpe ratio but ensure reasonable accuracy
        if accuracy_score < 0.5:  # Below random chance
            combined_score = -1.0
        else:
            combined_score = 0.7 * sharpe_score + 0.3 * (accuracy_score - 0.5) * 2
        
        # Log trial results
        trial.set_user_attr('mean_accuracy', cv_results['mean_accuracy'])
        trial.set_user_attr('std_accuracy', cv_results['std_accuracy'])
        trial.set_user_attr('mean_sharpe', cv_results['mean_sharpe'])
        trial.set_user_attr('std_sharpe', cv_results['std_sharpe'])
        trial.set_user_attr('num_features', len(selected_features))
        
        # Report intermediate value for pruning
        trial.report(combined_score, step=0)
        
        # Check if trial should be pruned
        if trial.should_prune():
            raise optuna.TrialPruned()
        
        return combined_score
        
    except Exception as e:
        logger.error(f"Trial failed: {str(e)}")
        return -1.0  # Return poor score for failed trials
    
    finally:
        # Clean up memory
        tf.keras.backend.clear_session()

print("✅ Objective function defined")
print("\nObjective function details:")
print("- Primary metric: Sharpe ratio (70% weight)")
print("- Secondary metric: Accuracy above 50% (30% weight)")
print("- Minimum accuracy threshold: 50% (random chance)")
print("- Cross-validation: Time series split")
print("- Pruning: MedianPruner for early stopping")

## 8. Run Hyperparameter Optimization

In [ ]:
def run_optimization():
    """
    Run the hyperparameter optimization.
    """
    print(f"🎯 Starting hyperparameter optimization for {TARGET_SYMBOL}")
    print(f"Trials: {N_TRIALS}, CV splits: {CV_SPLITS}, Timeout: {TIMEOUT}s")
    
    # Create study
    study_name = f"cnn_lstm_{TARGET_SYMBOL}_{datetime.now().strftime('%Y%m%d_%H%M%S')}"
    
    study = optuna.create_study(
        direction='maximize',
        sampler=TPESampler(seed=42),
        pruner=MedianPruner(n_startup_trials=5, n_warmup_steps=10),
        study_name=study_name
    )
    
    # Run optimization
    study.optimize(
        objective,
        n_trials=N_TRIALS,
        timeout=TIMEOUT,
        n_jobs=N_JOBS,
        show_progress_bar=True
    )
    
    return study

# Run the optimization
print("🚀 Starting optimization...")
study = run_optimization()

print(f"\n✅ Optimization completed!")
print(f"Number of finished trials: {len(study.trials)}")
print(f"Number of pruned trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}")
print(f"Number of complete trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}")

## 9. Analyze Optimization Results

In [ ]:
def analyze_results(study):
    """
    Analyze and visualize optimization results.
    """
    print("📊 Analyzing optimization results...")
    
    # Best trial
    best_trial = study.best_trial
    
    print(f"\n🏆 Best Trial Results:")
    print(f"Objective value: {best_trial.value:.6f}")
    print(f"Mean accuracy: {best_trial.user_attrs.get('mean_accuracy', 'N/A'):.4f}")
    print(f"Mean Sharpe ratio: {best_trial.user_attrs.get('mean_sharpe', 'N/A'):.4f}")
    print(f"Number of features: {best_trial.user_attrs.get('num_features', 'N/A')}")
    
    print(f"\n🔧 Best Hyperparameters:")
    for key, value in best_trial.params.items():
        print(f"  {key}: {value}")
    
    # Top 5 trials
    print(f"\n🥇 Top 5 Trials:")
    top_trials = sorted(study.trials, key=lambda t: t.value or -999, reverse=True)[:5]
    
    for i, trial in enumerate(top_trials, 1):
        print(f"\n{i}. Trial {trial.number}:")
        print(f"   Objective: {trial.value:.6f}")
        print(f"   Accuracy: {trial.user_attrs.get('mean_accuracy', 'N/A'):.4f}")
        print(f"   Sharpe: {trial.user_attrs.get('mean_sharpe', 'N/A'):.4f}")
        print(f"   Lookback: {trial.params.get('lookback_window')}")
        print(f"   Features: {trial.user_attrs.get('num_features', 'N/A')}")
    
    # Parameter importance
    print(f"\n📈 Parameter Importance:")
    try:
        importance = optuna.importance.get_param_importances(study)
        for param, imp in sorted(importance.items(), key=lambda x: x[1], reverse=True)[:10]:
            print(f"  {param}: {imp:.4f}")
    except Exception as e:
        print(f"  Could not calculate importance: {e}")
    
    return best_trial

# Analyze results
best_trial = analyze_results(study)

## 10. Visualize Optimization Results

In [ ]:
def plot_optimization_results(study):
    """
    Create visualization plots for optimization results.
    """
    print("📊 Creating optimization visualizations...")
    
    # Create subplots
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    fig.suptitle(f'Hyperparameter Optimization Results - {TARGET_SYMBOL}', fontsize=16)
    
    # 1. Optimization history
    ax1 = axes[0, 0]
    values = [trial.value for trial in study.trials if trial.value is not None]
    ax1.plot(values, 'b-', alpha=0.7)
    ax1.plot(np.cummax(values), 'r-', linewidth=2, label='Best value')
    ax1.set_xlabel('Trial')
    ax1.set_ylabel('Objective Value')
    ax1.set_title('Optimization History')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # 2. Parameter correlation
    ax2 = axes[0, 1]
    completed_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]
    
    if len(completed_trials) > 10:
        # Lookback vs Objective
        lookbacks = [t.params['lookback_window'] for t in completed_trials]
        objectives = [t.value for t in completed_trials]
        ax2.scatter(lookbacks, objectives, alpha=0.6)
        ax2.set_xlabel('Lookback Window')
        ax2.set_ylabel('Objective Value')
        ax2.set_title('Lookback Window vs Performance')
        ax2.grid(True, alpha=0.3)
    
    # 3. Accuracy vs Sharpe
    ax3 = axes[1, 0]
    accuracies = [t.user_attrs.get('mean_accuracy', 0) for t in completed_trials]
    sharpes = [t.user_attrs.get('mean_sharpe', 0) for t in completed_trials]
    
    if accuracies and sharpes:
        scatter = ax3.scatter(accuracies, sharpes, c=[t.value for t in completed_trials], 
                            cmap='viridis', alpha=0.7)
        ax3.set_xlabel('Mean Accuracy')
        ax3.set_ylabel('Mean Sharpe Ratio')
        ax3.set_title('Accuracy vs Sharpe Ratio')
        ax3.grid(True, alpha=0.3)
        plt.colorbar(scatter, ax=ax3, label='Objective Value')
    
    # 4. Feature count distribution
    ax4 = axes[1, 1]
    feature_counts = [t.user_attrs.get('num_features', 0) for t in completed_trials]
    
    if feature_counts:
        ax4.hist(feature_counts, bins=20, alpha=0.7, edgecolor='black')
        ax4.set_xlabel('Number of Features')
        ax4.set_ylabel('Frequency')
        ax4.set_title('Feature Count Distribution')
        ax4.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    # Save the plot
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    plot_filename = f"optimization_results_{TARGET_SYMBOL}_{timestamp}.png"
    plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
    print(f"📁 Plot saved as: {plot_filename}")

# Create visualizations
plot_optimization_results(study)

## 11. Save Optimization Results

In [ ]:
def save_optimization_results(study, best_trial):
    """
    Save optimization results to files.
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Create results directory
    results_dir = "optimization_results"
    os.makedirs(results_dir, exist_ok=True)
    
    # Save best parameters
    best_params_file = os.path.join(results_dir, f"best_params_{TARGET_SYMBOL}_{timestamp}.json")
    with open(best_params_file, 'w') as f:
        json.dump({
            'symbol': TARGET_SYMBOL,
            'timestamp': timestamp,
            'objective_value': best_trial.value,
            'mean_accuracy': best_trial.user_attrs.get('mean_accuracy'),
            'mean_sharpe': best_trial.user_attrs.get('mean_sharpe'),
            'num_features': best_trial.user_attrs.get('num_features'),
            'best_params': best_trial.params
        }, indent=2)
    
    print(f"✅ Best parameters saved to: {best_params_file}")
    
    # Save all trials data
    trials_data = []
    for trial in study.trials:
        trial_data = {
            'trial_number': trial.number,
            'value': trial.value,
            'state': str(trial.state),
            'params': trial.params,
            'user_attrs': trial.user_attrs
        }
        trials_data.append(trial_data)
    
    trials_file = os.path.join(results_dir, f"all_trials_{TARGET_SYMBOL}_{timestamp}.json")
    with open(trials_file, 'w') as f:
        json.dump(trials_data, indent=2)
    
    print(f"✅ All trials saved to: {trials_file}")
    
    # Save study object
    study_file = os.path.join(results_dir, f"study_{TARGET_SYMBOL}_{timestamp}.pkl")
    with open(study_file, 'wb') as f:
        pickle.dump(study, f)
    
    print(f"✅ Study object saved to: {study_file}")
    
    # Create summary report
    summary_file = os.path.join(results_dir, f"optimization_summary_{TARGET_SYMBOL}_{timestamp}.txt")
    with open(summary_file, 'w') as f:
        f.write(f"Hyperparameter Optimization Summary\n")
        f.write(f"=====================================\n\n")
        f.write(f"Symbol: {TARGET_SYMBOL}\n")
        f.write(f"Timestamp: {timestamp}\n")
        f.write(f"Total trials: {len(study.trials)}\n")
        f.write(f"Completed trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE])}\n")
        f.write(f"Pruned trials: {len([t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED])}\n\n")
        
        f.write(f"Best Trial Results:\n")
        f.write(f"  Objective value: {best_trial.value:.6f}\n")
        f.write(f"  Mean accuracy: {best_trial.user_attrs.get('mean_accuracy', 'N/A'):.4f}\n")
        f.write(f"  Mean Sharpe ratio: {best_trial.user_attrs.get('mean_sharpe', 'N/A'):.4f}\n")
        f.write(f"  Number of features: {best_trial.user_attrs.get('num_features', 'N/A')}\n\n")
        
        f.write(f"Best Hyperparameters:\n")
        for key, value in best_trial.params.items():
            f.write(f"  {key}: {value}\n")
    
    print(f"✅ Summary report saved to: {summary_file}")
    
    return {
        'best_params_file': best_params_file,
        'trials_file': trials_file,
        'study_file': study_file,
        'summary_file': summary_file
    }

# Save results
print("💾 Saving optimization results...")
saved_files = save_optimization_results(study, best_trial)

print("\n🎉 Hyperparameter optimization completed successfully!")
print("\n📁 Files saved:")
for file_type, filepath in saved_files.items():
    print(f"  {file_type}: {filepath}")

## 12. Test Best Model

In [ ]:
def test_best_model(best_params):
    """
    Train and test model with best parameters using GPU.
    """
    print(f"🧪 Testing best model with optimized parameters...")
    
    # Prepare data with best parameters
    X, y, selected_features, scaler = prepare_training_data(
        TARGET_SYMBOL, 
        all_indicators[TARGET_SYMBOL], 
        rcs, 
        prices, 
        best_params
    )
    
    print(f"Data prepared: {X.shape}, Features: {len(selected_features)}")
    
    # Split data
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X[:split_idx], X[split_idx:]
    y_train, y_test = y[:split_idx], y[split_idx:]
    
    # Create and train best model with GPU
    with tf.device('/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'):
        model = create_optimized_model((X.shape[1], X.shape[2]), best_params)
        
        early_stopping = EarlyStopping(
            monitor='val_loss',
            patience=best_params['patience'],
            restore_best_weights=True,
            verbose=1
        )
        
        print("Training best model on GPU...")
        print(f"🎮 Using device: {'/GPU:0' if tf.config.list_physical_devices('GPU') else '/CPU:0'}")
        
        history = model.fit(
            X_train, y_train,
            validation_split=0.15,
            epochs=best_params['epochs'],
            batch_size=best_params['batch_size'],
            callbacks=[early_stopping],
            verbose=1
        )
    
    # Evaluate
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
    y_pred_proba = model.predict(X_test, verbose=0).flatten()
    
    print(f"\n✅ Best Model Results:")
    print(f"Test Accuracy: {test_acc:.4f}")
    print(f"Test Loss: {test_loss:.4f}")
    print(f"Features used: {len(selected_features)}")
    
    # Plot training history
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 2, 1)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title(f'{TARGET_SYMBOL} - Best Model Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.grid(True)
    
    plt.subplot(1, 2, 2)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title(f'{TARGET_SYMBOL} - Best Model Training Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.grid(True)
    
    plt.tight_layout()
    plt.show()
    
    # Save best model
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    model_filename = f"best_model_{TARGET_SYMBOL}_{timestamp}.h5"
    model.save(model_filename)
    print(f"\n💾 Best model saved as: {model_filename}")
    
    return model, selected_features, scaler

# Test the best model
best_model, best_features, best_scaler = test_best_model(best_trial.params)

print(f"\n🎯 Hyperparameter optimization completed successfully!")
print(f"\n📊 Summary:")
print(f"  Best objective value: {best_trial.value:.6f}")
print(f"  Best accuracy: {best_trial.user_attrs.get('mean_accuracy', 'N/A'):.4f}")
print(f"  Best Sharpe ratio: {best_trial.user_attrs.get('mean_sharpe', 'N/A'):.4f}")
print(f"  Optimal features: {len(best_features)}")
print(f"  Optimal lookback: {best_trial.params['lookback_window']}")

print(f"\n🚀 Ready for deployment with optimized hyperparameters!")

# Final GPU status check
print(f"\n🔍 Final GPU Status:")
print(f"GPU devices: {tf.config.list_physical_devices('GPU')}")
if tf.config.list_physical_devices('GPU'):
    print("✅ GPU was successfully used throughout hyperparameter optimization")
else:
    print("⚠️ CPU was used for hyperparameter optimization")

## Summary

This hyperparameter optimization notebook provides:

### ✅ **Comprehensive Optimization**
- **Model Architecture**: Conv1D filters, LSTM units, layer configuration
- **Training Parameters**: Learning rate, batch size, epochs, patience
- **Regularization**: Dropout rates, L1/L2 regularization
- **Data Processing**: Lookback window, feature selection methods
- **Trading Logic**: Confidence thresholds for signal generation

### 🎯 **Multi-Objective Optimization**
- **Primary**: Sharpe ratio (risk-adjusted returns)
- **Secondary**: Model accuracy above random chance
- **Constraints**: Minimum data requirements, performance thresholds

### 🔬 **Robust Validation**
- **Time Series Cross-Validation**: Respects temporal order
- **Early Pruning**: Stops poor trials early
- **Multiple Metrics**: Accuracy, Sharpe ratio, drawdown, win rate

### 📊 **Comprehensive Analysis**
- **Parameter Importance**: Identifies most influential parameters
- **Visualization**: Optimization history, parameter relationships
- **Results Export**: JSON, pickle, summary reports

### 🚀 **Production Ready**
- **Best Model Export**: Saves optimized model for deployment
- **Reproducible Results**: Saved parameters and random seeds
- **Memory Management**: Proper cleanup between trials

### 📈 **Next Steps**
1. **Apply optimized parameters** to other currency pairs
2. **Multi-symbol optimization** for portfolio strategies
3. **Online optimization** for adaptive parameters
4. **Ensemble optimization** combining multiple models

The optimization process systematically finds the best combination of hyperparameters to maximize risk-adjusted returns while maintaining model robustness.